In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
df = pd.read_csv('/kaggle/input/new-york-city-airbnb-open-data/AB_NYC_2019.csv')
df.head()

In [ ]:
# Target variable reviews_per_month reviews_per_month 
print(df.info())

In [ ]:
# converting categorical features

def code_myohe(data, feature):
    for i in data[feature].unique():
        if type(i)==type(1.0):
            data[feature+"="+str(i)]=(data[feature]==i).astype("float64");
        else:
            data[feature+"="+i]=(data[feature]==i).astype("float64");
        
code_myohe(df, "neighbourhood_group");
df.drop(['neighbourhood_group'], axis=1, inplace=True) 
code_myohe(df, 'room_type');
df.drop(['room_type'], axis=1, inplace=True) 


#def code_myohe2(data, feature):   
#    j=0;
#    for i in data[feature].unique():
#        data[feature+"_num"]=j
#        j++;
#    categories_num = data[feature].unique().count
#    
#    for i in data[feature+"_num"].unique():
#        data[feature+"_e"+i]=j
#        j++;
        
lr = "last_review";
df[lr+"_weekday"] = pd.to_datetime(df[lr], errors='coerce').dt.strftime("%a");

In [ ]:


code_myohe(df, lr+"_weekday");
df.drop([lr+"_weekday"], axis=1, inplace=True);
df[lr+'_month']= pd.to_datetime(df[lr], errors='coerce').dt.strftime("%b");
code_myohe(df, lr+'_month');
df.drop([lr+'_month'], axis=1, inplace=True) ;
df[lr+'_day']= pd.to_datetime(df[lr], errors='coerce').dt.strftime("%d");
code_myohe(df, lr+'_day');
df.drop([lr+'_day'], axis=1, inplace=True) ;
df[lr+"_year"]= pd.to_datetime(df[lr], errors='coerce').dt.strftime("%Y");
code_myohe(df, lr+'_year');
df.drop([lr+'_year'], axis=1, inplace=True) ;
df.drop([lr], axis=1, inplace=True);
df.drop(["name"], axis=1, inplace=True);
df.drop(["host_name"], axis=1, inplace=True);
df.drop(["neighbourhood"], axis=1, inplace=True);
df.drop(["id"], axis=1, inplace=True);
df.drop(["host_id"], axis=1, inplace=True);

print(df.info())

In [ ]:
y = df['reviews_per_month']
y = y.fillna(-1)
y_classific = y>=(y.mean());
df.drop(['reviews_per_month'], axis=1, inplace=True)

from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

X_train, X_holdout, y_train, y_holdout = train_test_split(df.values, y, test_size=0.3, random_state=17)

print(X_train.shape)
print(X_holdout.shape)

y_regress_train, y_regress_holdout = y_train, y_holdout;
y_train, y_holdout = (y_train>=(y.mean())), (y_holdout>=(y.mean()));

tree = DecisionTreeClassifier(max_depth=6, random_state=17)
knn = KNeighborsClassifier(n_neighbors=10)
rndForest = RandomForestClassifier()
grdBost = GradientBoostingClassifier()

tree.fit(X_train, y_train)
knn.fit(X_train, y_train)
rndForest.fit(X_train, y_train)
grdBost.fit(X_train, y_train)



In [ ]:
from sklearn.metrics import accuracy_score

zero_pred = np.zeros(shape=(y_holdout.shape));
print(accuracy_score(y_holdout, zero_pred)) # 0.636

tree_pred = tree.predict(X_holdout)
print(accuracy_score(y_holdout, tree_pred)) # 0.900

knn_pred = knn.predict(X_holdout)
print(accuracy_score(y_holdout, knn_pred)) # 0.826

rndForest_pred = rndForest.predict(X_holdout)
print(accuracy_score(y_holdout, rndForest_pred)) # 0.906

grdBost_pred = grdBost.predict(X_holdout)
print(accuracy_score(y_holdout, grdBost_pred)) # 0.908

In [ ]:

from sklearn.linear_model import LogisticRegression, LinearRegression

logRegress = LogisticRegression();

logRegress.fit(X_train, y_train)

In [ ]:


logRegress_pred = logRegress.predict(X_holdout)
print(accuracy_score(y_holdout, logRegress_pred)) # 0.895


In [ ]:
#network
from keras import models
from keras import layers

data = np.concatenate((X_train, X_holdout), axis=0)
targets = np.concatenate((y_train, y_holdout), axis=0)

def vectorize(sequences, dimension = 78):
 results = np.zeros((len(sequences), dimension))
 for i, sequence in enumerate(sequences):
  results[i, sequence] = 1
 return results

In [ ]:
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

In [ ]:
model = models.Sequential()
# Input - Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(78, )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()
# compiling the model
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)
results = model.fit(
 train_x, train_y,
 epochs= 14,
 batch_size = 500,
 validation_data = (test_x, test_y)
)
print(results.history.keys())
print("Test-Accuracy:", np.mean(results.history["val_accuracy"]))